# 2.4 Modelado Dimensional Básico\n\n- Objetivo: comprender dimensiones y hechos, granularidad y cardinalidad.\n- Prerrequisitos: tablas dim_* y fact_* creadas.\n- Ejercicios: validar unicidad, densidad actividad, gaps inventario.\n- Reto: mini-dimensión fecha y días sin ventas.\n- Errores comunes: mezclar granularidades, claves poco claras, sobre-normalizar dimensiones pequeñas.

Elementos del dataset:
- Dimensiones: clientes, productos, regiones.
- Hechos: ventas, inventario, suscripciones.
Claves sustitutas vs naturales, granularidad de la tabla de hechos (venta por línea).

In [ ]:
-- Granularidad: validar que cada venta_id es único
SELECT venta_id, COUNT(*) AS repeticiones
FROM dbo.fact_ventas
GROUP BY venta_id
HAVING COUNT(*) > 1;

In [ ]:
-- Cardinalidad: clientes vs ventas
SELECT COUNT(*) AS clientes, (SELECT COUNT(*) FROM dbo.fact_ventas) AS filas_ventas
FROM dbo.dim_clientes;

🟢 Ejercicio: Calcular razón (filas ventas / clientes) para densidad de actividad promedio.

In [ ]:
SELECT CAST(v.filas AS DECIMAL(10,2))/c.clientes AS ventas_por_cliente
FROM (SELECT COUNT(*) AS filas FROM dbo.fact_ventas) v
CROSS JOIN (SELECT COUNT(*) AS clientes FROM dbo.dim_clientes) c;

🟠 Ejercicio: Detectar productos sin registros en inventario (dim vs hecho).

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_inventario fi ON p.producto_id = fi.producto_id
WHERE fi.producto_id IS NULL;

🔴 Reto: Derivar una 'mini dimensión de fecha' on-the-fly con CTE (lista de fechas min->max) y unir para detectar días sin ventas.

In [ ]:
-- Ejemplo idea (ajustar según motor):
-- WITH fechas AS (
--   SELECT MIN(fecha) AS fmin, MAX(fecha) AS fmax FROM dbo.fact_ventas
-- ), serie AS (
--   SELECT fmin AS fecha FROM fechas
--   UNION ALL
--   SELECT DATEADD(DAY,1,fecha) FROM serie s JOIN fechas f ON s.fecha < f.fmax
-- ) SELECT fecha FROM serie EXCEPT SELECT DISTINCT fecha FROM dbo.fact_ventas;

Errores comunes: mezclar granularidades; no definir claves claras; intentar normalizar en exceso dimensiones pequeñas.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.